 # Opis notatnika
 Zmierzamy do końca analizy danych, które zostały nam udostępnione. Ten krok dodaje jeszcze więcej informacji do naszego wyjściowego zbioru. Tym razem sprawdzimy między innymi to, czy opóźnienia lotów zależne są od trasy czy warunków pogodowych.

 Zanim jednak do tego przejdziemy, należy, podobnie jak w poprzednich krokach, skonfigurować odpowiednio notatnik.

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import sys
sys.path.append('../')

pd.set_option("display.max_colwidth", 3000)
pd.set_option("display.max_columns", None)

%matplotlib inline

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
from config.DB_connect import username, passwd, hostname, db_name

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [ ]:
url = f"postgresql://{username}:{passwd}@{hostname}:{port}/{db_name}"
engine = create_engine(url, echo=False)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [ ]:
def read_sql_table(table_name):
    return pd.read_sql(f"SELECT * FROM {table_name}", engine)

 Tutaj zaczytaj zapisaną wcześniej ramkę danych `flight_df` do zmniennej o takiej samej nazwie

In [ ]:
flight_df = pd.read_csv("../data/processed/flight_02.csv", index_col=None)

 # Wzbogacenie o `airport_list`
 Wczytaj do obszaru roboczego tabelę `airport_list` używając procedury `read_sql_table`. Wykonaj poniższe ćwiczenia:
 1. Sprawdź, czy klucz `origin_airport_id` jest unikalny, tj. nie ma dwóch takich samych wartości w kolumnie `origin_airport_id`,
 1. Jeżeli duplikaty występują, usuń je w najdogodniejszy dla Ciebie sposób.
 1. Jeśli duplikaty nie występują, złącz ramki `airport_list_df` wraz z aktualną `flight_df`, używając kolumny `origin_airport_id` oraz złączenia typu `LEFT JOIN`. Z ramki `airport_list_df` interesuje nas dodanie kolumny `origin_city_name`.
 1. Dodatkowo dokonaj jeszcze raz złączenia ramki `flight_df` z `airport_list_df`, tym razem jednak złącz kolumnę `destination_airport_id` wraz z `origin_airport_id`. Podobnie jak wcześniej, interesuje nas kolumna `origin_city_name`, jedank ona powinna zostać wyświetlona jako `destination_city_name`

 Tutaj wczytaj ramkę `airport_list_df`

In [ ]:
airport_list_df = read_sql_table("airport_list")

airport_list_df.rename(
    columns={
        "name": "origin_name",
        "display_airport_name": "origin_airport_name"
    },
    inplace=True,
)

 Tutaj sprawdż, czy występują duplikaty dla kolumny `origin_airport_id`

In [ ]:
airport_list_df[airport_list_df["origin_airport_id"].duplicated()]

In [ ]:
airport_list_df.drop(columns=["id"], inplace=True)

 Tutaj usuń duplikaty - jeśli występują

'origin_airport_id' nie zawiera duplikatów

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `origin_airport_id`

In [ ]:
flight_df = flight_df.merge(
    airport_list_df,
    how="left",
    on="origin_airport_id"
)

 Tutaj dokonaj złączenia ramki `flight_df` oraz `airport_list_df` używając `destination_airport_id`

In [ ]:
airport_list_df.rename(
    columns={
        "origin_airport_id": "dest_airport_id",
        "origin_airport_name": "destination_airport_name",
        "origin_city_name": "destination_city_name",
        "origin_name": "destination_name",
    },
    inplace=True,
)

In [ ]:
flight_df = flight_df.merge(
    airport_list_df,
    how="left",
    on="dest_airport_id",
)

### Sprawdzenie
Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
assert (
    "origin_city_name" in flight_df.columns
), "Brak kolumny `origin_city_name` w ramce flight_df"
assert (
    "destination_city_name" in flight_df.columns
), "Brak kolumny `destination_city_name` w ramce flight_df"

flight_df_expected_rows_amount = 6922924
assert (
    flight_df.shape[0] == flight_df_expected_rows_amount
), "Ups, zwiększyła się liczba wierszy..."

 ## Analiza według lotnisk oraz tras
 Wykonaj poniższe polecenia:
 1. Wyznacz lotniska, z których **odlatywało** najwięcej samolotów. Wynik zapisz do ramki `top_airports_origin_df`.
 1. Wyznacz lotnika, na których najwięcej lotów **się kończyło**. Wynik zapisz do ramki `top_airports_destination_df`.
 1. Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki `top_route_df`.
 1. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki `least_route_delays_df`.
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki `top_route_delays_df`.

 Tutaj wyznacz ramkę `top_airports_origin_df`

In [ ]:
top_airports_origin_df = flight_df.groupby(
    "origin_airport_id", as_index=None).count()['id']

 Tutaj wyznacz ramkę `top_airports_destination_df`

In [ ]:
top_airports_destination_df = flight_df.groupby(
    "dest_airport_id", as_index=None).count()['id']

 ### Sprawdzenie dla `top_airport_origin`

In [ ]:
top_airports_origin_head = (
    top_airports_origin_df.sort_values(ascending=False).head().to_list()
)
top_airports_origin_head = tuple(top_airports_origin_head)
top_airports_origin_head_expected = (387620, 327647, 294944, 245160, 228415)

assert (
    top_airports_origin_head == top_airports_origin_head_expected
), f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_origin_head_expected} otrzymano: {top_airports_origin_head}"

### Sprawdzenie dla `top_airport_destination`

In [ ]:
top_airports_destination_head = (
    top_airports_destination_df.sort_values(ascending=False).head().to_list()
)
top_airports_destination_head = tuple(top_airports_destination_head)
top_airports_destination_head_expected = (
    387542, 327169, 294568, 244898, 227917)

assert (
    top_airports_destination_head == top_airports_destination_head_expected
), f"Nie zgadza się top 5 wierszy, oczekiwano wyników: {top_airports_destination_head_expected} otrzymano: {top_airports_destination_head}"

 1. Wyznacz najczęściej uczęszczaną trasę, wynik zapisz do ramki `top_route_df`.


In [ ]:
top_route_df = flight_df.groupby(
    ['origin_airport_id', 'dest_airport_id'], as_index=None)['id'].agg('count')
top_route_df.rename(columns={'id': 'route_count'}, inplace=True)
top_route_df.sort_values(by=['route_count'], inplace=True, ascending=False)

 1. Przy założeniu, że reprezentatywna liczba lotów na trasie wynosi ponad 500, wyznacz dodatkowo top 10:
     - tras z **najmniejszym odsetkiem opóźnień**, wynik zapisz do ramki `least_route_delays_df`.
     - tras z **największym odsetkiem opóźnień**, wynik zapisz do ramki `top_route_delays_df`.

In [ ]:
# dodawanie informacji o ilości lotów do flight_df

flight_df = pd.merge(
    left=flight_df,
    right=top_route_df,
    on=['origin_airport_id', 'dest_airport_id']
)
flight_df.sort_values(by=['route_count'], inplace=True, ascending=False)

flight_df.shape[0]

In [ ]:
least_route_delays_df = flight_df.loc[flight_df['route_count'] > 500].sort_values(
    by=['dep_delay'], ascending=True)
least_route_delays_df.head()

In [ ]:
least_route_delays_df.shape[0]

In [ ]:
top_route_delays_df = flight_df.loc[flight_df['route_count'] > 500].sort_values(
    by=['dep_delay'], ascending=False)
top_route_delays_df.head()

In [ ]:
top_route_delays_df.shape[0]

 # Wzbogacenie o dane pogodowe
 Używając procedury `read_sql_table`, wczytaj tabelę `airport_weather` do ramki `airport_weather_df`. Następnie wykonaj następujące polecenia:
 1. Pozostaw w ramce tylko następujące kolumny: `['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']`.
 1. Połącz ramki `airport_list_df` wraz z `airport_weather_df` po odpowiedniej kolumnie używając takiego złączenia, aby w wyniku usunąć te wiersze (lotniska), które nie posiadają danych pogodowych. Dodatkowo, upewnij się, że zostanie tylko dodana kolumna `origin_airport_id`.

 Tutaj wczytaj ramkę `airport_weather`

In [ ]:
airport_weather_df = read_sql_table("airport_weather")
airport_weather_df.columns

 Tutaj oczyść ramkę `airport_weather_df` z nadmiarowych kolumn

In [ ]:
airport_weather_df.drop(
    columns=[
        "id",
        "wt18",
        "tavg",
        "tmin",
        "wdf2",
        "wdf5",
        "wsf2",
        "wsf5",
        "wt01",
        "wt08",
        "wt02",
        "wt03",
        "wt04",
        "wt09",
        "wt06",
        "wt05",
        "pgtm",
        "wt10",
        "wesd",
        "sn32",
        "sx32",
        "psun",
        "tsun",
        "tobs",
        "wt07",
        "wt11",
    ],
    inplace=True,
)

In [ ]:
airport_weather_df.columns

In [ ]:
airport_list_df.columns

 Tutaj połącz ramki `airport_list_df` oraz `airport_weather_df` aktualizując `airport_weather_df`

In [ ]:
airport_weather_df = pd.merge(
    left=airport_list_df,
    right=airport_weather_df,
    left_on="destination_name",
    right_on="name",
)
airport_weather_df

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (43394, 12)
airport_weather_df_shape = airport_weather_df.shape

assert (
    airport_weather_df_expected_shape == airport_weather_df_shape
), f"Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}"

 ## Połączenie `airport_weather_df` oraz `flight_df`
 W celu złączenia ramek `airport_weather_df` oraz `flight_df` wykonaj następujące kroki:
 1. w ramce `aiport_weather_df` występuje kolumna `date`, zrzutuj ją na typ `DATETIME`.
 1. w ramce `flight_df` należy stworzyć nową kolumnę o nazwie `date`. W tym celu:
 	- złącz kolumny `month`, `day_of_month` oraz `year` razem, użyj następującego formatu daty: `YYYY-MM-DD`.
 	- zrzutuj kolumnę `date` na typ `DATETIME`.
 1. złącz ramki używając odpowiedniego klucza, wynik złączenia zapisz do ramki `flight_df`. Uzyj złącznia typu `LEFT JOIN`.

 > Dlaczego istotne jest zachowanie typów przy złączeniu?

W trakcie pracy możesz posłużyć się następującymi artykułami z `LMS`:
 - `Python - analiza danych > Dzień 6 - Pandas > Merge`
 - `Python - analiza danych > Dzień 6 - Pandas > Praca z datetime`
 - Dokumentacje metody `to_datetime`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
 - Dostępne formaty dat: [klik](https://www.programiz.com/python-programming/datetime/strftime) - sekcja `Format Code List`

 Tutaj zrzutuj kolumnę `date` na `DATETIME` w ramce `airport_weather_df`

In [ ]:
airport_weather_df["date"] = pd.to_datetime(airport_weather_df["date"])
airport_weather_df['date'].info()

 Tutaj stwórz kolumnę `date` w ramce `flight_df`. Pamiętaj, aby była ona również typu `DATETIME`

In [ ]:
flight_df['date'] = pd.to_datetime(
    dict(year=flight_df.year, month=flight_df.month, day=flight_df.day_of_month))
flight_df['date'].info()

In [ ]:
flight_df.columns

In [ ]:
airport_weather_df.columns

In [ ]:
airport_weather_df.drop(columns=["name"], inplace=True)

 Tutaj złącz tabele `airport_weather_df` oraz `flight_df`

In [ ]:
flight_df = flight_df.merge(airport_weather_df, how='left', on=['date',
                                                                'dest_airport_id',
                                                                'destination_airport_name',
                                                                'destination_city_name',
                                                                'destination_name'],
                            )

In [ ]:
flight_df.columns

In [ ]:
airport_weather_df.rename(
    columns={
        "dest_airport_id": "origin_airport_id",
        "destination_airport_name": "origin_airport_name",
        "destination_city_name": "origin_city_name",
        "destination_name": "origin_name",
    },
    inplace=True,
)

In [ ]:
airport_weather_df.columns

In [ ]:
flight_df = flight_df.merge(airport_weather_df,
                            how='left',
                            on=["date", "origin_airport_id",
                                "origin_airport_name",
                                "origin_city_name",
                                "origin_name",
                                ],
                            suffixes=("_dest", "_origin"),
                            )

In [ ]:
flight_df.columns

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_rows_amount = 6922924
assert (
    flight_df.shape[0] == flight_df_expected_rows_amount
), "Ups, zmieniła się liczba wierszy..."

 # Praca samodzielna
 Używając `flight_df` zbadaj następujące hipotezy:
 1. Opady śniegu w lotnisku wylotowym wpływają na **wielkość** opóźnienia lotów (kolumna `snow`).
 1. Wielkość pokrywy śnieżnej wpływa na **wielkość** opóźnienia lotów (kolumna `snwd`).
 1. Temperatura maksymalna wpływa na **wielkość** opóźnienia lotów (kolumna `tmax`).
 W każdym ćwiczeniu pamiętaj o uwzględnieniu tylko tych zjawisk atmosferycznych, które były zaobserwowane (`>`). Przy wykonywaniu tego zadania masz pełną dowolność.

> **Wskazówka:**  
> Pamiętaj o tym, aby każda analiza była porównywalna, tj. dokonana przy podobnych założeniach.

 ## Analiza dla kolumny `snow`

 ### Określenie statystyk opisowych dla kolumny `snow`

In [ ]:
flight_df_snow = flight_df.loc[flight_df['snow_origin'] > 0][[
    'dep_delay', 'snow_origin']]

In [ ]:
# flight_df_snow.corr().style.background_gradient(cmap='seismic')

In [ ]:
plt.scatter(flight_df_snow.snow_origin, flight_df_snow.dep_delay, alpha=0.5)
plt.xlabel = "ilość opadów śniegu"
plt.ylabel = "czas opóżnienia odlotów"
plt.show()

 ## Analiza dla kolumny `snwd`

 ### Określenie statystyk opisowych dla kolumny `snwd`

In [ ]:
flight_df_snwd = flight_df.loc[flight_df['snwd_origin'] > 0][[
    'dep_delay', 'snwd_origin']]
# flight_df_snwd.corr().style.background_gradient(cmap='seismic')

In [ ]:
plt.scatter(flight_df_snwd.snwd_origin, flight_df_snwd.dep_delay, alpha=0.5)
plt.xlabel = "grubość zalegającego śniegu"
plt.ylabel = "czas opóżnienia odlotów"
plt.show()

 ## Analiza dla kolumny `tmax`

 ### Określenie statystyk opisowych dla kolumny `tmax`

In [ ]:
flight_df_tmax = flight_df[['dep_delay', 'tmax_origin']]
# flight_df_tmax.corr().style.background_gradient(cmap='seismic')

In [ ]:
plt.scatter(flight_df_tmax.tmax_origin, flight_df_tmax.dep_delay, alpha=0.5)
plt.xlabel = "temperatura"
plt.ylabel = "czas opóżnienia odlotów"
plt.show()

# Podsumowanie
W tej części warsztatu dokonaliśmy kompleksowej analizy posiadanego zbioru danych. Eksploracja
pozwoliła nam na zapoznanie się z cechami charakterystycznymi lotów - wiemy już, które 
zmienne mogą mieć wpływ na opóźnienia lotów, a które nie. Co warto podkreślić, skupiliśmy się na wielu
aspektach tej analizy, co otwiera potencjalnie również inne możliwości dalszej pracy nad tą bazą.

W tym momencie przejdziemy do kolejnego kroku, w którym, na podstawie tej analizy, przygotujemy 
system raportowy. Zanim jednak stworzymy dashboard, potrzebujemy zaktualizować naszą bazę danych.